# Set up X and Y

In [1]:
import pandas as pd
df = pd.read_csv('tSNE_3D_xkcd_noDates.csv')
display(df.head())

X = df[["x1","x2","x3","comicID"]]
display(X.head())
y = df[["gmmCluster"]]
display(y.head())
X.describe()

,Unnamed: 0,comicID,x1,x2,x3,gmmCluster,url
0,832,833,-19.124754,-4.478358,12.616282,0,"=hyperlink(""https://xkcd.com/833/"")"
1,946,947,-16.081493,-5.278377,19.839954,0,"=hyperlink(""https://xkcd.com/947/"")"
2,1917,1918,-14.891591,5.949551,8.775877,0,"=hyperlink(""https://xkcd.com/1918/"")"
3,948,949,-17.840237,7.002065,19.548286,0,"=hyperlink(""https://xkcd.com/949/"")"
4,1916,1917,-17.134342,7.867988,14.172421,0,"=hyperlink(""https://xkcd.com/1917/"")"


,x1,x2,x3,comicID
0,-19.124754,-4.478358,12.616282,833
1,-16.081493,-5.278377,19.839954,947
2,-14.891591,5.949551,8.775877,1918
3,-17.840237,7.002065,19.548286,949
4,-17.134342,7.867988,14.172421,1917


,gmmCluster
0,0
1,0
2,0
3,0
4,0


,x1,x2,x3,comicID
count,2015.000000,2015.000000,2015.000000,2015.000000
mean,0.059887,0.441721,-0.180581,1008.000000
std,12.101639,9.388049,12.208811,581.824716
min,-27.041980,-22.028961,-31.256489,1.000000
25%,-8.723360,-6.300513,-9.008545,504.500000
50%,0.300769,0.673480,0.300259,1008.000000
75%,8.841077,7.077730,8.768749,1511.500000
max,29.238255,24.600662,27.680058,2015.000000


# Test train split and set up the model

In [61]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf 
from sklearn.metrics import classification_report, confusion_matrix

lb = preprocessing.LabelBinarizer()
y=lb.fit_transform(y)

print(y.shape)
print(X.shape)

X=np.array(X)
y=np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

#Thanks to: https://stackoverflow.com/questions/43076609/how-to-calculate-precision-and-recall-in-keras
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

model = Sequential()

model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',precision, recall])
epochs=0
bestmodel=model
for i in range(10):
    model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=0)
    epochs+=50
    score = model.evaluate(X_test, y_test, batch_size=128)  
    precision=score[2]
    recall=score[3]
    f1=2*(precision*recall)/(precision+recall)
    print(epochs,score)
    print(round(f1,3))
    y_pred=model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    
    print('Confusion Matrix')
    print(confusion_matrix(np.argmax(lb.inverse_transform(y_test),axis=1), y_pred))
#     print('Classification Report')
#     print(classification_report(lb.classes_, y_pred, target_names=lb.classes_))

(2015, 10)
(2015, 4)
202/202 [==============================] - 1s 4ms/step
50 [12.310030918310185, 0.18316832037255315, 0.11454892697015612, 0.2973548707985642]
0.165
Confusion Matrix
[[ 1  0  0 11  0  0  0  0  0  0]
 [ 1 18  0 13  0  0  0  0  0  0]
 [ 0 17  0  1  1  0  0  0  0  0]
 [ 0  0  0 17  0  0  0  0  0  0]
 [ 0  0  0 24  1  1  0  0  0  0]
 [ 4  0  0 23  0  0  0  0  0  0]
 [ 1  1  0 17  0  0  0  0  0  0]
 [ 0  7  0  5  0  0  0  0  0  0]
 [ 0 23  0  0  0  0  0  0  0  0]
 [ 0  1  0 14  0  0  0  0  0  0]]
202/202 [==============================] - 0s 48us/step
100 [2.223116164160247, 0.17326732658513702, 0.11210553097252798, 0.42395124842624854]
0.177
Confusion Matrix
[[ 1  0  0  0 11  0  0  0  0  0]
 [ 1  0  0  1 30  0  0  0  0  0]
 [ 0  0  0  0 19  0  0  0  0  0]
 [ 0  0  0  0 16  1  0  0  0  0]
 [ 0  0  0  0 25  1  0  0  0  0]
 [ 0  0  0  0 19  8  0  0  0  0]
 [ 0  0  0  0 18  1  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0 22  0  0  0  1  0]
 [ 0  0  0  0 15  0  

In [59]:
result=model.predict(X_test)

for i in range (10):
    print(X_test)
    print(np.around(result[i],2))


             x1         x2         x3  comicID
1198 -14.531134  18.492931  15.477062      176
526   12.742083  -1.505909 -13.019718     1823
393   11.206589  -9.385802  -3.430337     1399
1407   5.653759  -3.309086  20.235306     1139
433   -1.891783  -0.407094   9.820305     1808
1720   1.331170 -13.977160 -16.733555     1185
1090  -1.240907  16.024220  -0.199780     1713
429   -1.976729  -3.295764   7.371072      612
1799   2.012441  -9.617510 -18.089872      424
530   11.084832 -17.567995 -18.661040     1281
1727   3.497717  -9.532889 -24.484074     2002
1453   9.621297  14.549671  16.420013     1825
1293 -10.847492   9.395681   3.056922       36
1618  14.868760   6.302911  -6.878973     1029
1022   0.652428   8.919335   2.124826      597
678  -13.842903   5.760849  -0.850882     1130
1128  -9.073868   6.548285  -2.527691     1494
124  -16.333021   3.950084  17.255825     1770
1473  -5.701252   5.659096  20.879568       29
462   -2.928600  -6.434484  -3.546549      224
1856   8.3542